In [1]:
import sqlite3
import pandas as pd

import sys
sys.path.append("../")
from controller import *

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\andre\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\andre\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package rslp to
[nltk_data]     C:\Users\andre\AppData\Roaming\nltk_data...
[nltk_data]   Package rslp is already up-to-date!


In [2]:
decretos_sen = load_senado_decrees('../Senado API files/', '../radar_db')
classes_sen=get_classes("../supervised NLP/Classification/classificacao senado")
classes_sen_names = [c[0] for c in classes_sen]
decretos_sen = multilabel_classify_decrees(decretos_sen, classes_sen)
decretos_sen = get_df_senado(decretos_sen)
decretos_sen = decretos_sen.set_index("numero ato")

decretos_minist = load_ministerio_decrees('../radar_db')
classes_minist=get_classes("../supervised NLP/Classification/classificacao referenda")
classes_minist_names = [c[0] for c in classes_minist]
decretos_minist = multilabel_classify_decrees(decretos_minist, classes_minist)
decretos_minist = get_df_ministerio(decretos_minist)
decretos_minist['cod_ident_ato'] = [int(d.replace(" ", "/").split("/")[1].replace('.', '')) for d in decretos_minist['cod_ident_ato']]
decretos_minist = decretos_minist.set_index("cod_ident_ato")

10135 is not classified by Senado.
10125 is not classified by Senado.
10021 is not classified by Senado.
9205 is not classified by Senado.
8420 is not classified by Senado.


In [3]:
decretos_minist

,full_text,referenda
cod_ident_ato,,
10095,Dispõe sobre o Comitê Consultivo de Nanotecnol...,"[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0]"
10093,"Altera o Decreto nº 8.642 , de 19 de janeiro ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0]"
10091,Promulga o Acordo entre o Governo da República...,"[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
10090,Promulga o Acordo entre o Governo da República...,"[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
10094,Dispõe sobre o Comitê Interministerial de Tecn...,"[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0]"
...,...,...
3332,"PROMULGA O PROTOCOLO AO TRATADO DE AMIZADE, CO...","[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
3337,PRORROGA A VIGÊNCIA DOS RESTOS A PAGAR QUE ESP...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0]"
3328,"ALTERA O DECRETO N. 2.889, DE 21/12/1998, QUE ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0]"


In [4]:
nodes = pd.read_csv('graph csv/nodes_gephi_v3.csv')
nodes['Label'] = [l.replace(" ", "/").split("/")[1].replace('.', '') for l in nodes['Label']]

In [5]:
def enrich_with_classes(df):
    df["Senate"] = ""
    df["Cabinets"] = ""
    df["Label"] = df["Label"].astype(int)
    df = df.set_index("Label")

    for i in df.index:
        for j, label in enumerate(decretos_sen.loc[i]["classificação senado"]):
            if label == 1 and j < len(classes_sen_names):
                df.loc[i, "Senate"] += (classes_sen_names[j] + ";")

        for j, label in enumerate(decretos_minist.loc[i]["referenda"]):
            if label == 1 and j < len(classes_minist_names):
                df.loc[i, "Cabinets"] += (classes_minist_names[j] + "; ")
    return df

## Degree centrality

In [6]:
# Top degree centrality decrees
degree = nodes.sort_values(["degree", "outdegree"], ascending=[False, False]).head(5)
degree = enrich_with_classes(degree)
degree

,Id,timeset,indegree,outdegree,degree,modularity_class,Eccentricity,closnesscentrality,harmonicclosnesscentrality,betweenesscentrality,Senate,Cabinets
Label,,,,,,,,,,,,
10086,DEC 10.086/2019,NaN,0,135,135,0,21,0.242086,0.417237,0.000000,,Presidência;
6944,DEC 6.944/2009,NaN,112,11,123,15,3,0.692308,0.796296,0.000235,"Assuntos internos, cargos e comissões, Estado;",Economia e planejamento;
10179,DEC 10.179/2019,NaN,0,115,115,8,8,0.542857,0.753966,0.000000,,Presidência;
9917,DEC 9.917/2019,NaN,0,97,97,51,15,0.274556,0.564485,0.000000,"Assuntos internos, cargos e comissões, Estado;",Presidência;
9757,DEC 9.757/2019,NaN,0,81,81,75,4,0.831858,0.921099,0.000000,,Presidência;


In [7]:
# Top 100 degree centrality decrees that contain the word "PAC" in the abstract
indices = np.array(nodes.sort_values(["degree", "outdegree"], ascending=[False, False]).head(100)["Label"]).astype(int)
lookup_degree = nodes.copy()
lookup_degree["Label"] = lookup_degree["Label"].astype(int)
lookup_degree.set_index("Label", inplace=True)

pac = pd.DataFrame(decretos_sen.loc[indices]["full_text"])
pac["degree"] = ""
pac["modularity_class"] = ""
pac_df = pac[pac['full_text'].str.contains(' PAC')]

for i in pac_df.index:
    pac_df.loc[i]["degree"] = lookup_degree.loc[i]["degree"]
    pac_df.loc[i]["modularity_class"] = lookup_degree.loc[i]["modularity_class"]

pac_df

,full_text,degree,modularity_class
numero ato,,,
8765,DISCRIMINA AÇÕES DO PROGRAMA DE ACELERAÇÃO DO ...,45,132
8509,DISCRIMINA AÇÕES DO PROGRAMA DE ACELERAÇÃO DO ...,45,132
8267,DISCRIMINA AÇÕES DO PROGRAMA DE ACELERAÇÃO DO ...,45,132
8227,DISCRIMINA AÇÕES DO PROGRAMA DE ACELERAÇÃO DO ...,45,132
8206,DISCRIMINA AÇÕES DO PROGRAMA DE ACELERAÇÃO DO ...,45,132
9154,DISCRIMINA AÇÕES DO PROGRAMA DE ACELERAÇÃO DO ...,44,132
8926,DISCRIMINA AÇÕES DO PROGRAMA DE ACELERAÇÃO DO ...,44,132
8659,DISCRIMINA AÇÕES DO PROGRAMA DE ACELERAÇÃO DO ...,44,132
8286,DISCRIMINA AÇÕES DO PROGRAMA DE ACELERAÇÃO DO ...,44,132


## Betweeness

In [8]:
betweenness = nodes.sort_values(["betweenesscentrality", "outdegree"], ascending=[False, False]).head(5)
betweenness = enrich_with_classes(betweenness)
betweenness

,Id,timeset,indegree,outdegree,degree,modularity_class,Eccentricity,closnesscentrality,harmonicclosnesscentrality,betweenesscentrality,Senate,Cabinets
Label,,,,,,,,,,,,
6188,DEC 6.188/2007,NaN,13,7,20,144,9,0.290076,0.377480,0.000405,"Assuntos internos, cargos e comissões, Estado;",Presidência; Economia e planejamento;
7462,DEC 7.462/2011,NaN,6,13,19,144,10,0.277778,0.360185,0.000333,"Assuntos internos, cargos e comissões, Estado;",Ciência e tecnologia; Economia e planejamento;
6944,DEC 6.944/2009,NaN,112,11,123,15,3,0.692308,0.796296,0.000235,"Assuntos internos, cargos e comissões, Estado;",Economia e planejamento;
6521,DEC 6.521/2008,NaN,18,4,22,356,10,0.235772,0.295886,0.000233,"Assuntos internos, cargos e comissões, Estado;",Presidência; Economia e planejamento;
5783,DEC 5.783/2006,NaN,3,5,8,144,8,0.325581,0.434949,0.000218,"Assuntos internos, cargos e comissões, Estado;",Economia e planejamento;


## Most central decrees by area

In [9]:
def count_nodes(decretos, nodes, metric, classification, limit, classes_names):
    central_nodes = nodes[nodes[metric] != 0.0].sort_values([metric, "outdegree"], ascending=[False, False])[:limit]
    counters = np.zeros(len(classes_names)+1)
    for d in central_nodes[["Label", metric]].values:
        counters = np.add(counters, decretos.loc[int(d[0])][classification])
        
    for c in range(len(counters)):
        try:
            print(classes_names[c], counters[c]/limit*100)
        except:
            print("Unknown: ", counters[c]/limit*100)
            print("\n")

In [10]:
print("BETWEENESS:")
print("Senado")
count_nodes(decretos_sen, nodes, "betweenesscentrality", "classificação senado", 100, classes_sen_names)
print("Referenda")
count_nodes(decretos_minist, nodes, "betweenesscentrality", "referenda", 100, classes_minist_names)

print("CLOSENESS:")
print("Senado")
count_nodes(decretos_sen, nodes, "closnesscentrality", "classificação senado", 100, classes_sen_names)
print("Referenda")
count_nodes(decretos_minist, nodes, "closnesscentrality", "referenda", 100, classes_minist_names)

print("DEGREE:")
print("Senado")
count_nodes(decretos_sen, nodes, "degree", "classificação senado", 100, classes_sen_names)
print("Referenda")
count_nodes(decretos_minist, nodes, "degree", "referenda", 100, classes_minist_names)

BETWEENESS:
Senado
Saúde 0.0
Relações Exteriores 1.0
Meio ambiente 0.0
Educação, cultura e esporte 0.0
Segurança Pública e Defesa 0.0
Trabalho e Previdência 4.0
Agricultura, pecuária e pesca 0.0
Ciência, tecnologia e comunicações 1.0
Social 0.0
Indústria, comércio, turismo, transporte/transporte de mercadorias 0.0
Economia, planejamento e sistema financeiro 1.0
Assuntos internos,  cargos e comissões, Estado 88.0
Tributos 6.0
Minas e Energia 0.0
Justiça e direitos 2.0
Unknown:  2.0


Referenda
Saúde 0.0
Relações Exteriores 2.0
Meio ambiente 1.0
Educação, cultura e esporte 5.0
Justiça e Segurança 16.0
Trabalho e Previdência 6.0
Transporte/transporte de mercadorias 0.0
Agricultura, pecuária e pesca 3.0
Ciência e tecnologia 5.0
Social 4.0
Presidência 18.0
Economia e planejamento 96.0
Indústria, comércio, obras públicas, turismo 4.0
Unknown:  0.0


CLOSENESS:
Senado
Saúde 3.0
Relações Exteriores 8.0
Meio ambiente 1.0
Educação, cultura e esporte 5.0
Segurança Pública e Defesa 7.0000000000000